# [BEGAN = EBGAN + WGAN](https://arxiv.org/pdf/1703.10717.pdf) [local-paper](http://localhost:8888/notebooks/Dropbox/Paper/1703-10717.pdf) [reddit](https://www.reddit.com/r/MachineLearning/comments/633jal/r170310717_began_boundary_equilibrium_generative/) [code](https://github.com/wiseodd/generative-models/tree/master/GAN/boundary_equilibrium_gan)

<img src="img/BEGAN.png"/>


초짜 대학원생의 입장에서 이해하는 BEGAN 
[1](http://jaejunyoo.blogspot.com/2017/04/began-boundary-equilibrium-gan-1.html)
[2](http://jaejunyoo.blogspot.com/2017/04/began-boundary-equilibrium-gan-2.html)

BEGAN: STATE OF THE ART GENERATION OF FACES WITH GENERATIVE ADVERSARIAL NETWORKS [heuritech](https://blog.heuritech.com/2017/04/11/began-state-of-the-art-generation-of-faces-with-generative-adversarial-networks/)
[reddit](https://www.reddit.com/r/MachineLearning/comments/64rayf/r_began_state_of_the_art_generation_of_faces_with/)

Code by 
[carpedm20](https://github.com/carpedm20/BEGAN-tensorflow)
[artcg](https://github.com/artcg/BEGAN)
[mokemokechicken](https://github.com/mokemokechicken/keras_BEGAN)

<img src="img/BEGAN - Wasserstein distance lower bound for auto-encoders.png"/>

<img src="img/BEGAN - GAN objective 1.png"/>

<img src="img/BEGAN - GAN objective 2.png"/>

<img src="img/BEGAN - Equilibrium.png"/>

<img src="img/BEGAN - Boundary Equilibrium GAN.png"/>

<img src="img/BEGAN - Convergence measure.png"/>

In [ ]:
# https://github.com/wiseodd/generative-models/blob/master/GAN/boundary_equilibrium_gan/began_tensorflow.py

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

mb_size = 32
X_dim = 784
z_dim = 64
h_dim = 128
lr = 1e-3
m = 5
lam = 1e-3
gamma = 0.5
k_curr = 0

try:
    mnist = input_data.read_data_sets("data/MNIST/", one_hot=True)
except:
    mnist = input_data.read_data_sets('../../Data/MNIST', one_hot=True)

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

X = tf.placeholder(tf.float32, shape=[None, X_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])
k = tf.placeholder(tf.float32)

D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
D_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
D_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

G_W1 = tf.Variable(xavier_init([z_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))
G_W2 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]
theta_D = [D_W1, D_W2, D_b1, D_b2]

def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

def G(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

def D(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    X_recon = tf.matmul(D_h1, D_W2) + D_b2
    return tf.reduce_mean(tf.reduce_sum((X - X_recon)**2, 1))

G_sample = G(z)

D_real = D(X)
D_fake = D(G_sample)

D_loss = D_real - k*D_fake
G_loss = D_fake

D_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=theta_G)

with tf.Session() as sess:
    tf.global_variables_initializer().run()

    if not os.path.exists('out/'):
        os.makedirs('out/')

    i = 0
    for it in range(1000000):
        X_mb, _ = mnist.train.next_batch(mb_size)
        _, D_real_curr = sess.run([D_solver, D_real], feed_dict={X: X_mb, z: sample_z(mb_size, z_dim), k: k_curr})
        _, D_fake_curr = sess.run([G_solver, D_fake], feed_dict={X: X_mb, z: sample_z(mb_size, z_dim)})
        k_curr = k_curr + lam * (gamma*D_real_curr - D_fake_curr)

        if it % 1000 == 0:
            measure = D_real_curr + np.abs(gamma*D_real_curr - D_fake_curr)
            print('Iter-{}; Convergence measure: {:.4}'.format(it, measure))
            samples = sess.run(G_sample, feed_dict={z: sample_z(16, z_dim)})
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)